In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from tensorflow.keras.layers import GlobalAveragePooling1D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import TFBertForSequenceClassification
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.optimizers import adam_v2

In [3]:
tf.config.run_functions_eagerly(True)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = "/content/drive/My Drive/Capstone/data/depression_pre.csv"
dp = pd.read_csv(path)

In [6]:
path2 = "/content/drive/My Drive/Capstone/data/depression_post.csv"
dp_post = pd.read_csv(path2)

In [8]:
pre = dp.drop(columns=['Unnamed: 0', 'Post URL', 'Original Content', 'Saved','ID', 'year', 'month'])

In [9]:
post = dp_post.drop(columns=['Unnamed: 0', 'Post URL', 'Original Content', 'Saved','ID', 'year', 'month'])

In [10]:
pre_txt = pre[["Post Text"]]
post_txt = post[["Post Text"]]

In [11]:
pre_txt['label'] = 'pre'
post_txt['label'] = 'post'

<ipython-input-11-26faaad642e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['label'] = 'pre'
<ipython-input-11-26faaad642e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['label'] = 'post'


In [12]:
pre_txt['Post Text'] = pre_txt['Post Text'].values
post_txt['Post Text'] = post_txt['Post Text'].values

<ipython-input-12-714712031f77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'] = pre_txt['Post Text'].values
<ipython-input-12-714712031f77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'] = post_txt['Post Text'].values


In [13]:
pre_txt['Post Text'].fillna('', inplace=True)
pre_txt.dropna(subset=['Post Text'], inplace=True)

<ipython-input-13-7f9de0dda555>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'].fillna('', inplace=True)
<ipython-input-13-7f9de0dda555>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt.dropna(subset=['Post Text'], inplace=True)


In [14]:
post_txt['Post Text'].fillna('', inplace=True)
post_txt.dropna(subset=['Post Text'], inplace=True)

<ipython-input-14-60cd906aa43a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'].fillna('', inplace=True)
<ipython-input-14-60cd906aa43a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt.dropna(subset=['Post Text'], inplace=True)


In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [18]:
pre_txt['Post Text'] = pre_txt['Post Text'].apply(remove_punctuations)
post_txt['Post Text'] = post_txt['Post Text'].apply(remove_punctuations)

<ipython-input-18-ccfe400b5707>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'] = pre_txt['Post Text'].apply(remove_punctuations)
<ipython-input-18-ccfe400b5707>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'] = post_txt['Post Text'].apply(remove_punctuations)


In [19]:
pre_txt['Post Text'] = pre_txt['Post Text'].replace('\n','', regex=True)
post_txt['Post Text'] = post_txt['Post Text'].replace('\n','', regex=True)

<ipython-input-19-a13db772377b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_txt['Post Text'] = pre_txt['Post Text'].replace('\n','', regex=True)
<ipython-input-19-a13db772377b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_txt['Post Text'] = post_txt['Post Text'].replace('\n','', regex=True)


In [20]:
post_txt['Post Text'].iloc[1]

'Im never going to get better why the fuck should I even try'

In [21]:
# Function to tokenize and encode text
def tokenize_and_encode(text, label):
    text = text.numpy().decode()  # Convert symbolic tensor to string
    tokens = tokenizer.encode_plus(text,
                                   max_length=128,  # Adjust max_length as needed
                                   truncation=True,
                                   padding='max_length',
                                   add_special_tokens=True,
                                   return_attention_mask=True,
                                   return_token_type_ids=True)
    return {'input_ids': tokens['input_ids'],
            'token_type_ids': tokens['token_type_ids'],
            'attention_mask': tokens['attention_mask']}, label


In [23]:
# Tokenize and encode data
pre_covid_tokens = [(text, 0) for text in pre_txt['Post Text']]  # Encode 'pre' as 0
post_covid_tokens = [(text, 1) for text in post_txt['Post Text']]  # Encode 'post' as 1

# Combine datasets
all_tokens = pre_covid_tokens + post_covid_tokens

# Shuffle dataset
import random
random.shuffle(all_tokens)
# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_generator(lambda: all_tokens,
                                         output_types=(tf.string, tf.int64),
                                         output_shapes=((), ()))

# Tokenize and encode data within the dataset using tf.py_function
def tokenize_encode_wrapper(text, label):
    return tf.py_function(tokenize_and_encode, inp=[text, label], Tout=(tf.int32, tf.int64))

dataset = dataset.map(tokenize_encode_wrapper)

# Define model configuration
model_config = {
    "num_labels": 2,  # Number of output labels (pre-COVID, post-COVID)
    "output_attentions": False,
    "output_hidden_states": False
}


# Load pre-trained BERT model for sequence classification
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", **model_config)

# Compile the model
bert_model.compile(optimizer='adam',  # Use string identifier for optimizer
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

# Train the model
bert_model.fit(dataset.shuffle(len(all_tokens)).batch(8), epochs=3)




All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Attempt to convert a value ({'input_ids': [101, 2059, 2008, 2711, 5363, 2000, 2735, 2009, 8380, 7737, 1998, 7566, 2055, 4933, 2008, 3084, 2032, 6517, 5685, 2043, 2057, 2202, 2256, 3268, 2111, 2360, 2339, 2134, 2102, 2027, 3713, 2138, 2053, 2303, 2003, 2412, 2183, 2000, 4952, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def tokenize_and_encode(text, label):
  tokens = tokenizer.encode_plus(text.numpy().decode(),
                                 max_length=128,
                                 truncation=True,
                                 padding='max_length',
                                 add_special_tokens=True,
                                 return_attention_mask=True,
                                 return_token_type_ids=True,
                                 return_tensors='tf')
  return {'input_ids': tokens['input_ids'][0],
            'token_type_ids': tokens['token_type_ids'][0],
            'attention_mask': tokens['attention_mask'][0]}, label

In [ ]:
pre_tokens = [(text, 0) for text in pre_txt['Post Text']]  # Encode 'pre' as 0
post_tokens = [(text, 1) for text in post_txt['Post Text']]  # Encode 'post' as 1

In [ ]:
all_tokens = pre_tokens + post_tokens

In [ ]:
texts = [text for text, _ in all_tokens]
labels = [label for _, label in all_tokens]

In [ ]:
labels = tf.constant(labels)

In [ ]:
import random
random.shuffle(all_tokens)

In [ ]:
dataset = tf.data.Dataset.from_generator(lambda: all_tokens,
                                         output_types=(tf.string, tf.int64),
                                         output_shapes=((), ()))

In [ ]:
def tokenize_encode_wrapper(text, label):
    return tf.py_function(tokenize_and_encode, inp=[text, label], Tout=(tf.int32, tf.int64))

dataset = dataset.map(tokenize_encode_wrapper)

In [ ]:
model_config = {
    "num_labels": 2,  # Number of output labels (pre-COVID, post-COVID)
    "output_attentions": False,
    "output_hidden_states": False
}

In [ ]:
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", **model_config)

# Compile the model
bert_model.compile(optimizer='adam',  # Use string identifier for optimizer
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

# Train the model
bert_model.fit(dataset.shuffle(len(all_tokens)).batch(32), epochs=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Attempt to convert a value ({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101,  1045,  2123,  1521,  1056,  2113,  2065, 10334,  2842,
        2064, 14396,  2000,  2023,  2021,  2823,  1045,  4299,  1045,
        2001,  2074,  2730,  1045,  2123,  1521,  1056,  2215,  2000,
        3102,  2870,  2138,  1045,  2514,  2066,  2026, 12954,  2052,
        2228,  2009,  1521,  1055,  2014,  6346,  2008,  1045,  1521,
        1049,  2025,  2105,  4902,  1045,  2113,  2009,  2876,  1521,
        1056,  2022,  2026,  3291,  4902,  2021,  2012,  1996,  2168,
        2051,  1045,  2123,  1521,  1056,  2113,  2129,  2000,  4863,
        2009,  1045,  2074,  2876,  1521,  1056,  2215,  2014,  2000,
        2514,  2066,  2016,  2071,  2031,  2589,  2242,  2000,  2562,
        2033,  2105,  2074,  2245,  1045,  1521,  1040,  2681,  20

In [ ]:
#pre_tokens = [(text, label) for text, label in zip(pre_txt['Post Text'], pre_txt['label'])]
#post_tokens = [(text, label) for text, label in zip(post_txt['Post Text'], post_txt['label'])]


#pre_covid tokenize_and_encode(text) for text in pre_txt['Post Text']]

#post_tokens = [tokenize_and_encode(text) for text in post_txt['Post Text']]

In [ ]:
pre_df = tf.data.Dataset.from_generator(lambda: (tokenize_and_encode(text, label) for text, label in pre_tokens),
                                                   output_types=({'input_ids': tf.int32,
                                                                  'token_type_ids': tf.int32,
                                                                  'attention_mask': tf.int32},
                                                                 tf.int64),
                                                   output_shapes=({'input_ids': (128,),
                                                                   'token_type_ids': (128,),
                                                                   'attention_mask': (128,)},
                                                                  ()))

post_df = tf.data.Dataset.from_generator(lambda: (tokenize_and_encode(text, label) for text, label in post_tokens),
                                                   output_types=({'input_ids': tf.int32,
                                                                  'token_type_ids': tf.int32,
                                                                  'attention_mask': tf.int32},
                                                                 tf.int64),
                                                   output_shapes=({'input_ids': (128,),
                                                                   'token_type_ids': (128,),
                                                                   'attention_mask': (128,)},
                                                                  ()))